In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import numpy as np
import matplotlib.pyplot as plt
import torch 
from torch import nn
torch.set_default_dtype(torch.float64)
import torch.nn.functional as func

import copy
from tqdm.notebook import tqdm
import time
import random
import seaborn as sns
sns.set_theme()

from sklearn.neighbors import NearestNeighbors
from scipy.interpolate import griddata
from scipy.spatial.distance import pdist, squareform
from sklearn.metrics.pairwise import euclidean_distances
from IPython.display import clear_output

In [ ]:
plt.rcParams['text.usetex'] = True

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from Example1 import Mueller_System
from utils    import LangevinIntegrator,rL2,Model,Model_cpu,Solver,Metadynamics,Metadynamics_Extend

In [ ]:
SYS = Mueller_System(); EPS = 10;
LI  = LangevinIntegrator(dim=SYS.dim)

In [ ]:
def get_q_NN(X,model_name=-1): 
    if model_name!=-1: model.load_weights(model_name);
    return model.get_q(X).reshape(-1)
def replace_lines(file_name, line_num, text):
    lines = open(file_name, 'r').readlines()
    for k in range(len(line_num)): lines[line_num[k]] = text[k]
    out = open(file_name, 'w')
    out.writelines(lines)
    out.close()  
def get_q_FEM(X,m=200,mb=50,file_name='MU_2D.edp'):
    eps=EPS
    random_id = np.random.randint(1e8)
    file_name0 = file_name
    file_name  = file_name[:-4] + str(random_id) + file_name[-4:]
    ! cp $file_name0 $file_name
    np.savetxt("X"+str(random_id),X[:,:2])
    replace_lines(file_name,[0,10,22,23],["real eps="+str(eps)+";\n","int m=%d,mb=%d;\n"%(m,mb),
                                          "ifstream fin(\"X"+str(random_id) + "\");\n",
                                          "ofstream fout(\"q"+str(random_id) + "\");\n"])
    !FreeFem++ $file_name > FEM.log
    os.remove("X"+str(random_id))
    os.remove(file_name)
    q = np.loadtxt("q"+str(random_id)).reshape(-1)
    os.remove("q"+str(random_id))
    return q[:len(X)] # may output len(X)+1 values

# Read some data

In [ ]:
def get_X_A(n,w=2*SYS.sigma*np.sqrt(EPS)):
    X = np.random.uniform(SYS.A[0]-SYS.r,SYS.A[0]+SYS.r,(10*n,SYS.dim))
    X[:,1] = np.random.uniform(SYS.A[1]-SYS.r,SYS.A[1]+SYS.r,(10*n))
    X[:,2:] = np.random.uniform(-w,w,(10*n,SYS.dim-2))
    mask = SYS.IsInA(X)
    return X[mask][:n]
def get_X_B(n,w=2*SYS.sigma*np.sqrt(EPS)):
    X = np.random.uniform(SYS.B[0]-SYS.r,SYS.B[0]+SYS.r,(10*n,SYS.dim))
    X[:,1] = np.random.uniform(SYS.B[1]-SYS.r,SYS.B[1]+SYS.r,(10*n))
    X[:,2:] = np.random.uniform(-w,w,(10*n,SYS.dim-2))
    mask = SYS.IsInB(X)
    return X[mask][:n]
def get_min_V_2D():
    xx = np.linspace(-.8,-.3,1000)
    yy = np.linspace(1.2,1.7,1000)
    XX,YY  = np.meshgrid(xx,yy)
    x_list = np.concatenate([XX[:,:,None],YY[:,:,None]],axis=-1).reshape(-1,2)
    return SYS.get_V_2D(x_list).min()
def get_uniform_data(N,Vbar=130):
    sigma     = SYS.sigma*np.sqrt(EPS)
    data      = np.random.uniform(-2*sigma,2*sigma,(N*10,SYS.dim))
    data[:,0] = np.random.uniform(SYS.xrange[0],SYS.xrange[1],N*10)
    data[:,1] = np.random.uniform(SYS.yrange[0],SYS.yrange[1],N*10)
    mask      = (SYS.get_V_2D(data[:,:2])-get_min_V_2D())<Vbar;
    return data[mask][:N]

In [ ]:
X_u = get_uniform_data(int(1e5))
q_u = get_q_FEM(X_u);
def Error_Model(model): return rL2(q_u,model.get_q(X_u))
mask = abs(q_u-0.5)<0.2; X_u2 = X_u[mask]; q_u2 = q_u[mask];
def Error_Model2(model): return rL2(q_u2,model.get_q(X_u2))
print(X_u.shape,X_u2.shape)

X_A = get_X_A(5000)
X_B = get_X_B(5000)
print(X_A.shape,X_B.shape)
def E_AB(model): return np.sqrt(np.mean(model.get_q(X_A)**2))+np.sqrt(np.mean((1-model.get_q(X_B))**2))

In [ ]:
def mask_fn(X): return (SYS.get_V(X)-SYS.get_V(X).min())<130; 
def ShowQandSampledData(q_fn,mask_fn=mask_fn,xrange=SYS.xrange,yrange=SYS.yrange,dim=SYS.dim,nx=100,ny=100,
                states=[],titles=[],q_list=np.linspace(.1,.9,9),fig_name=None):
    xx     = np.linspace(xrange[0],xrange[1],nx)
    yy     = np.linspace(yrange[0],yrange[1],ny)
    XX,YY  = np.meshgrid(xx,yy)
    x_list = np.concatenate([XX[:,:,None],YY[:,:,None]],axis=-1).reshape(-1,2)
    x_list = np.hstack([x_list,np.zeros(dtype=np.float64,shape=(x_list.shape[0],dim-2))])
    mask   = mask_fn(x_list); 
    V      = SYS.get_V(x_list); V[~mask] = np.nan;
    num    = len(q_fn)
    fig,ax = plt.subplots(1,num,figsize=(4.5*num,4),constrained_layout=True)
    for k in range(num):
        q  = q_fn[k](x_list); q[~mask] = np.nan;
        c1 = ax[k].contour(XX,YY,q.reshape(XX.shape),q_list,colors='black',linestyles='solid',linewidths=2);  
        ax[k].clabel(c1, inline=1, fontsize=10)
        if len(states)>0 and len(states[k])>0: [ax[k].scatter(d[:,0],d[:,1],s=1) for d in states[k]]
        
        ax[k].contour(XX,YY,V.reshape(XX.shape),5,colors='grey',linestyles='dashed',linewidths=.7);  
        ax[k].add_artist(plt.Circle(SYS.A, SYS.r, color='k'))
        ax[k].add_artist(plt.Circle(SYS.B, SYS.r, color='k'))
        ax[k].text(SYS.A[0]-0.1, SYS.A[1]+0.15, '$A$', fontsize=20)
        ax[k].text(SYS.B[0]+0.1, SYS.B[1]-0.20, '$B$', fontsize=20)
        ax[k].set_xlabel('$x_1$', fontsize=18)
        if k==0: ax[k].set_ylabel('$x_2$', fontsize=18, rotation=0)
        ax[k].tick_params(axis="both", labelsize=10) 
        ax[k].set_xlim(xrange)
        ax[k].set_ylim(yrange)
        if len(titles)>0: ax[k].set_title(titles[k],fontsize=20)

    if fig_name is not None: plt.savefig(fig_name,dpi=300)
    plt.show()    

# Obtain a NN close to $q$ Using SL

In [ ]:
class Commitor_Solver_SL():
    def __init__(self,model): 
        self.model    = model
    def sample_batch(self,data,batch_size):
        idx = random.sample(range(len(data)),  min(batch_size,len(data)))
        return data[idx]
    def get_loss(self,data): 
        X,q  = data[:,:self.model.input_dim],data[:,-1]
        qNN  = self.model.get_q(X)
        loss = torch.mean((qNN-q)**2)
        return loss,loss 
    def train_model(self,data_train,data_test,batch_size,optimizer,n_steps,
                    scheduler=None,n_show_loss=100,terminal_condition=None,
                    error_model1=None,error_model2=None,use_tqdm=True):
        
        if use_tqdm: step_range = tqdm(range(n_steps))
        else: step_range = range(n_steps)
        loss_step = []
        for i_step in step_range:
            if i_step%n_show_loss==0:
                loss_train,loss_test = self.get_loss(data_train)[:-1],\
                                       self.get_loss(data_test)[:-1]

                def show_num(x): 
                    if abs(x)<100 and abs(x)>.01: return '%0.5f'%x
                    else: return '%0.2e'%x
                item1 = '%2dk'%np.int_(i_step/1000)
                item2 = 'Loss: '+' '.join([show_num(k) for k in loss_train])
                item3 = ' '.join([show_num(k) for k in loss_test])
                item4 = ''
                if error_model1 is not None:
                    item4 = 'Error: '+show_num(error_model1(self.model))
                    if error_model2 is not None:
                        item4 = item4 + ' '+show_num(error_model2(self.model))
                print(', '.join([item1,item2,item3,item4]))
                loss_step = loss_step + [i_step] + [float(k) for k in loss_train]\
                                                 + [float(k) for k in loss_train]
            
            data_batch = self.sample_batch(data_train,batch_size)
            loss       = self.get_loss(data_batch)[-1]
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            if scheduler is not None: scheduler.step()
            if terminal_condition is not None:
                if terminal_condition(self.model): return
        return loss_step

In [ ]:
X_SL = get_uniform_data(int(1e5))
q_SL = get_q_FEM(X_SL)
print(X_SL.shape,q_SL.shape)

In [ ]:
n         = 5;
model     = Model_cpu(input_dim=SYS.dim,num_hidden=2,hidden_dim=50,n=n)
solver_SL = Commitor_Solver_SL(model)

In [ ]:
data_train = np.hstack([X_SL,q_SL.reshape(-1,1)])
data_train = torch.tensor(data_train)
data_test  = data_train

optimizer = torch.optim.Adam(model.parameters(),lr=torch.tensor(1e-3))
solver_SL.train_model(data_train=data_train,data_test=data_test,batch_size=5000,
                      optimizer=optimizer,n_steps=int(5e4+1),n_show_loss=1000,use_tqdm=True,
                      error_model1=Error_Model,error_model2=E_AB,)
ShowQandSampledData([model.get_q,model.get_r],states=[[X_SL],[]])

torch.save(model.state_dict(), "saved_models/SL")

# Some Functions

In [ ]:
def down_sample(X):
    mask = SYS.IsInA(X) | SYS.IsInB(X)
    return X[~mask]
def get_train_test(X,coef,X_A,X_B,ratio=0.7):
    Xc         = np.hstack([X,coef.reshape(-1,1)])
    data_train = []
    data_test  = []
    for d,requires_grad in [[Xc,True],[X_A,False],[X_B,False]]:
        perm = np.random.permutation(len(d))
        d1 = d[perm[:int(len(d)*ratio)]]
        data_train.append(torch.tensor(d1,requires_grad=requires_grad))
        d2 = d[perm[int(len(d)*ratio):]]
        data_test.append(torch.tensor(d2,requires_grad=requires_grad))
    return data_train,data_test 

In [ ]:
#########     training details  ##########################
c1 = 1;  c2 = 1; learning_rate = 1e-4;                   #
##########################################################

In [ ]:
def show_distr(X,ax):
    ax.scatter(X[:,0],X[:,1],s=1);
    ax.set_xlim(SYS.xrange); ax.set_ylim(SYS.yrange)
    ax.set_xlabel(r'$x_1$'); ax.set_ylabel(r'$x_2$',rotation=1)
    ax.set_title('shape:'+str(X.shape))

# Sampling Scheme I and II

In [ ]:
n      = 10;
model  = Model_cpu(input_dim=SYS.dim,num_hidden=2,hidden_dim=50,n=n)
solver = Solver(model,q0=-5,q1=5)
meta   = Metadynamics(model=model,h=2,w=.003)

# start from SL NN

model.load_state_dict(torch.load("saved_models/SL"))
ShowQandSampledData([model.get_q,model.get_r],titles=[r'$q$',r'$r$'])

# run metadyanmics

meta.re_init()
meta.perform(dV=SYS.get_dV,x=SYS.A.reshape(1,-1),dt=1e-5,eps=EPS,
             N=int(1e6),N_add=500,show_freq=.5,use_tqdm=True,show_distr=show_distr)
meta.show_meta(show_distr)

# Sample data

def get_f(X): return -SYS.get_dV(X) - meta.get_dV(X)
X0  = np.repeat(SYS.A.reshape(1,-1),50,axis=0)
X   = LI.get_data(X0,get_f,eps=EPS,dt=1e-5,m=100,T0=1,T=2,use_tqdm=True);
X   = down_sample(X) 
fig,ax = plt.subplots(1,1,figsize=(3,3))
show_distr(X,ax)
plt.show()

V_add = meta.get_V(X)
coef  = np.exp(1/EPS*(V_add-V_add.max()))
coef  = coef/coef.mean()

# Further training

data_train,data_test = get_train_test(X,coef,X_A,X_B,ratio=.9)
for i in range(len(data_train)): print(data_train[i].shape,data_test[i].shape)
optimizer = torch.optim.Adam(model.parameters(),lr=torch.tensor(learning_rate))
solver.train_model(data_train=data_train,data_test=data_test,c1=c1,c2=c2,batch_size=5000,
                   optimizer=optimizer,n_steps=int(2e4+1),n_show_loss=1000,use_tqdm=True,
                   error_model1=Error_Model,error_model2=Error_Model2)
ShowQandSampledData([get_q_FEM,model.get_q],states=[[],[X]],
                    titles=['Ref.','Sampling Scheme I'])

torch.save(meta,'saved_models/SL_alg1_meta')
torch.save([X,coef],'meta_data/SL_alg1_X_coef')
torch.save(model.state_dict(), 'saved_models/SL_alg1_Model')

In [ ]:
n      = 10;
model  = Model_cpu(input_dim=SYS.dim,num_hidden=2,hidden_dim=50,n=n)
solver = Solver(model,q0=-5,q1=5)
meta   = Metadynamics_Extend(model=model,h=2,w=.003,eps=EPS)

# start from SL NN

model.load_state_dict(torch.load("saved_models/SL"))
ShowQandSampledData([model.get_q,model.get_r],titles=[r'$q$',r'$r$'])

# run metadyanmics

meta.re_init()
meta.perform(dV=SYS.get_dV,x=SYS.A.reshape(1,-1),dt=1e-5,eps=EPS,
             N=int(1e6),N_add=500,show_freq=.5,use_tqdm=True,show_distr=show_distr)
meta.show_meta(show_distr)

# Sample data

def get_f(X): return -SYS.get_dV(X) + meta.get_dF(X)/2
X0  = np.repeat(SYS.A.reshape(1,-1),50,axis=0)
X   = LI.get_data(X0,get_f,eps=EPS,dt=1e-5,m=100,T0=1,T=2,use_tqdm=True);
X   = down_sample(X) 
fig,ax = plt.subplots(1,1,figsize=(3,3))
show_distr(X,ax)
plt.show()

V_add = -meta.get_F(X)/2
coef  = np.exp(1/EPS*(V_add-V_add.max()))
coef  = coef/coef.mean()

# Further training

data_train,data_test = get_train_test(X,coef,X_A,X_B,ratio=.9)
for i in range(len(data_train)): print(data_train[i].shape,data_test[i].shape)
optimizer = torch.optim.Adam(model.parameters(),lr=torch.tensor(learning_rate))
solver.train_model(data_train=data_train,data_test=data_test,c1=c1,c2=c2,batch_size=5000,
                   optimizer=optimizer,n_steps=int(2e4+1),n_show_loss=1000,use_tqdm=True,
                   error_model1=Error_Model,error_model2=Error_Model2)
ShowQandSampledData([get_q_FEM,model.get_q],states=[[],[X]],
                    titles=['Ref.','Sampling Scheme II'])

torch.save(meta,'saved_models/SL_alg2_meta')
torch.save([X,coef],'meta_data/SL_alg2_X_coef')
torch.save(model.state_dict(), 'saved_models/SL_alg2_Model')

# Umbrella Sampling

In [ ]:
class UmbrellaSampling():
    def __init__(self,kappa,q,eps,model):
        self.L     = len(q);
        self.kappa = kappa;
        self.q     = q;
        self.eps   = eps;
        self.model = model;
    def get_phi(self,X,l): 
        return np.exp(-1/self.eps*self.kappa*(self.model.get_q(X)-self.q[l])**2).reshape(-1)
    def get_gradphi_phi_Parallel(self,X,q):
        qv,qx = self.model.get_q_qx(X)
        return -1/self.eps*self.kappa*2*(qv.reshape(-1,1)-q)*qx
    def get_data_Parallel(self,x0,get_dV,n=2,dt=1e-2,m=100,T0=1,T=10):
        x0   = np.repeat(x0,n,axis=0)
        q    = np.repeat(self.q,n,axis=0).reshape(-1,1)
        def f(X,q=q): return -get_dV(X)+self.eps*self.get_gradphi_phi_Parallel(X,q)
        data = LI.get_data(x0=x0,f=f,eps=self.eps,dt=dt,m=m,T0=T0,T=T)
        nL   = x0.shape[0]
        data = np.hstack([data[k*nL:k*nL+nL].reshape(self.L,-1) for k in range(int(data.shape[0]/nL))])
        return [k.reshape(-1,x0.shape[1]) for k in data]
    def get_z(self,data):
        F = np.zeros(dtype=np.float64,shape=(self.L,self.L))
        for i in range(self.L):
            sum_phi_i = 0
            for j in range(self.L):
                sum_phi_i = sum_phi_i + self.get_phi(data[i],j)
            for j in range(self.L):
                F[i,j] = np.mean(self.get_phi(data[i],j)/sum_phi_i)
        F_t = np.transpose(F)
        F_t = F_t - np.identity(self.L)
        F_t[-1]=1; b=np.zeros(self.L); b[-1]=1
        return np.linalg.solve(F_t,b)
    def get_X_coef(self,data):
        Z    = self.get_z(data); X = np.vstack(data);
        coef = []; 
        for l,k in enumerate(Z): coef=coef+[k/data[l].shape[0]*X.shape[0]]*data[l].shape[0]
        coef = np.array(coef)
        return X,coef

In [ ]:
n      = 10;
model  = Model_cpu(input_dim=SYS.dim,num_hidden=2,hidden_dim=50,n=n)
solver = Solver(model,q0=-5,q1=5)
model.load_state_dict(torch.load("saved_models/SL"))

In [ ]:
# model.load_state_dict(torch.load("Example1_data/SL")) 

q_list = np.linspace(0,1,10)
US   = UmbrellaSampling(kappa=3000,q=q_list,eps=EPS,model=model)
x0   = np.repeat(np.vstack([SYS.A,SYS.B]),[5,5],axis=0)
data = US.get_data_Parallel(x0,get_dV=SYS.get_dV,dt=1e-5,m=100,T0=3,T=8,n=1);

# ShowQandSampledData([model.get_q,model.get_q],states=[[X_u2],[]],titles=['NN','FEM'])
ShowQandSampledData([model.get_q,model.get_q],states=[[],[]],titles=['NN','NN'])
fig, ax = plt.subplots(1,2,figsize=(15,5));
[ax[0].hist(model.get_q(d),15) for d in data]
# [ax[1].hist(get_q_FEM(d),15) for d in data]
[ax[k].set_xlim([0,1]) for k in range(2)]
[ax[k].set_title(s,fontsize=15) for k,s in enumerate(['NN','FEM'])];
plt.show()

In [ ]:
X,coef = US.get_X_coef(data);
print(X.shape)
def down_sample2(X,coef):
    mask = SYS.IsInA(X) | SYS.IsInB(X)
    return X[~mask],coef[~mask]
X,coef = down_sample2(X,coef)

data_train,data_test = get_train_test(X,coef,X_A,X_B,ratio=.9)
for i in range(len(data_train)): print(data_train[i].shape,data_test[i].shape)
optimizer = torch.optim.Adam(model.parameters(),lr=torch.tensor(learning_rate))
solver.train_model(data_train=data_train,data_test=data_test,c1=c1,c2=c2,batch_size=5000,
                   optimizer=optimizer,n_steps=int(20000+1),n_show_loss=1000,use_tqdm=True,
                   error_model1=Error_Model,error_model2=Error_Model2)
ShowQandSampledData([get_q_FEM,model.get_q],states=[[],[X]],
                    titles=['Ref.','Umbrella Sampling']) 

np.savez("US_data/SL_US_data",data)
torch.save([X,coef],'US_data/SL_US_X_coef')
torch.save(model.state_dict(), 'saved_models/SL_US_Model')